<a href="https://colab.research.google.com/github/TranQuocViet236/Codie_in_blog_AnhKhanh/blob/master/LSTM_AKhanh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import os


PRE-PROCESSING

In [ ]:
filename = '/content/gdrive/MyDrive/ML/LSTM/wonderland.txt'
#convert lower texts to reduce data size
raw_text = open(filename).read().lower()
print(raw_text)

In [ ]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c,i) for i,c in enumerate(chars))
print(f'number of letters: {len(char_to_int)}')
print(char_to_int)

In [ ]:
import string
string.ascii_lowercase

chars_new = list(string.ascii_lowercase) + ['0', '.', ',', ' ', '!', '?', 'unk'  ]
chars_to_int = dict((v,k) for k,v in enumerate(chars_new))
int_to_chars = dict((k,v) for k,v in enumerate(chars_new))

print('character to int: ', chars_to_int)
print('int to character: ', int_to_chars)



In [ ]:
n_chars = len(raw_text)
n_vocab = len(chars_new)

print('Total characters: ', n_chars)
print('Total vocab: ', n_vocab)


After, standardizing texts, we have 13637 words and 33 character


In [ ]:
def _encode_sen(text):
  text = text.lower()
  sen_vec = []
  for let in text:
    if let in chars_new[:-1]:
      idx = chars_to_int[let]
    else:
      idx = chars_to_int['unk']
    sen_vec.append(idx)
  return sen_vec


In [ ]:
print(chars_new[:-1])

In [ ]:
print(char_to_int['q'])

In [ ]:
x_test = _encode_sen('Alice is a wonderful story.#')

In [ ]:
print(x_test)

In [ ]:
def _decode_sen(vec):
  text = []
  for i in vec:
    let  = int_to_chars[i]
    text.append(let)
  text = ''.join(text)
  return text
   
    

In [ ]:
_decode_sen(x_test)

In [ ]:
#prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX  = []
dataY = []

for i in range(0, n_chars - seq_length, 1):
  #Get 100 previous characters 
  seq_in = raw_text[i:i+ seq_length]
  #Get the character after that 100 character
  seq_out = raw_text[i+ seq_length]

  dataX.append(_encode_sen(seq_in))
  dataY.append(_encode_sen(seq_out[0]))
  n_patterns = len(dataX)
print('Total Patterns: ', n_patterns)


In [ ]:
# reshape X to be [samples, time steps, features]
X_train = numpy.reshape(dataX, (n_patterns, seq_length, 1))
#Nomalize 
X_train = X_train/ float(n_vocab)
#one hot encode the output variable 
y_train = np_utils.to_categorical(dataY)

print(f'X[sample, time steps, features] shape: {X_train.shape}')
print(f'Y shape: {y_train.shape}')

In [ ]:
print(type(X_train))
print(type(y_train))

Data visualize

In [ ]:
# import seaborn as sns
# import numpy as np
# import matplotlib.pyplot as plt
# import pandas as pd

# print(dataY)
# print(len(dataY))
# plt.figure(figsize = (10,5))
# sns.countplot((np.array(dataY)))
# plt.xticks(np.arange(32), np.array(chars_new))

Build a model to predict

In [ ]:
model = Sequential()
model.add(LSTM(256, input_shape= (X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=['acc'])
model.summary()

In [ ]:
filepath = 'weights-improvement-{epoch:02d}-{loss:.4f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max')
callback_list = [checkpoint]
model.fit(X_train, y_train, epochs = 5, batch_size = 128, validation_split=0.33, callbacks = callback_list, verbose = 1)


In [ ]:
import numpy as np
base_word = 'Alice was beginning to get very tired of sitting by her sister on the bank'

def _predict_let(text, len_sen = 1):
  text_for = []
  for i in range(len_sen):
    x_input = np.array(_encode_sen(text)[-100:])/float(n_vocab)
    if x_input.shape[0] < 100:
      x_input = np.concatenate((np.zeros(100-x_input.shape[0]), x_input), axis = 0)
    x_input = np.expand_dims(np.expand_dims(x_input, -1), 0)
    y_prob = model.predict(x_input)
    y_let = int_to_chars[np.argmax(y_prob, axis=1)[0]]
    text = text + y_let
  return text[len_sen:]
_predict_let(base_word,100)